In [109]:
import math

In [110]:
class item_set_and_num:
    def __init__(self,item_set:set,num):
        self.item_set = item_set
        self.num = num
class item_sets:
    def __init__(self):
        self.item_sets = []
    def __getitem__(self,item_set:set):
        for idx,item in enumerate(self.item_sets):
            if item.item_set == item_set:
                return item
        return None
    def __len__(self):
        return len(self.item_sets)
    def __iter__(self):
        self.index =0
        return self
    def __next__(self):
        if self.index<len(self):
            self.index += 1
            return self.item_sets[self.index-1]
        else:
            raise StopIteration
    #根据项集作为下标增加
    def append(self,item_set:set,num=0):
        self.item_sets.append(item_set_and_num(item_set,num))
    #根据项集作为下标删除
    def pop(self,item_set:set):
        for idx,item in enumerate(self.item_sets.copy()):
            if item.item_set == item_set:
                self.item_sets.pop(idx)
    #过滤掉小于最小支持度的项集
    def filter(self,min_sup):
        del_num=0
        for idx,item_set in enumerate(self.item_sets.copy()):
            if item_set.num<min_sup:
                self.item_sets.pop(idx-del_num)
                del_num+=1
    def __str__(self) -> str:
        for item in self.item_sets:
            print(f"{item.item_set}:{item.num}")
        return ''


$L_k为频繁k项集，C_k为候选项集$

In [111]:
data = [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]
# data =[['A','C','D'],['B','C','E'],['A','B','C','E'],['B','E']]
# data =[[1,2,5], [2,4], [2,3], [1,2,4], [1,3], [2,3], [1,3], [1,2,3,5], [1,2,3]]
min_sup = 2
L_k = item_sets()
for item in data:
    for item_set in item:
        item_set = set({item_set})
        if L_k[item_set]==None:
            L_k.append(item_set,1)
        else:
            L_k[item_set].num+=1
L_k.filter(min_sup)

$求取C_k：当L_{k-1}中任意两个项集交集的元素个数为k时为候选k项集$

In [112]:
def get_C_k(L_k:item_sets):
    C_k = item_sets()
    for item1 in L_k.item_sets:
        for item2 in L_k.item_sets:
            new_item_set = item1.item_set|item2.item_set
            if len(new_item_set)==len(item1.item_set)+1:
                if C_k[new_item_set]==None:
                    C_k.append(new_item_set)
    return C_k      


$C_k 中的一个候选项集的某个k-1项子集不是L_{k-1}中的元素时，则将它从C_k中删除。$     
$等价于： C_k 中的一个候选项集的所有k-1项子集都是L_{k-1}中的元素，$     
$即L_{k-1}中是C_k中某个候选项集的子集的数目等于这个候选项集的k-1项子集的数目，否则将它从C_k中删除。$



In [113]:
def cut(C_k:item_sets,L:item_sets):
    C_k_1 = item_sets()
    for item in C_k:
        t = 0
        for item1 in L:
            if item1.item_set.issubset(item.item_set):
                t+=1
        base=len(item.item_set)
        if t==math.factorial(base)/(math.factorial(base-1)):
            C_k_1.append(item.item_set)
    return C_k_1

$计算C_k中每个k项集的支持数，并删除小于最小支持书的k项集$

In [114]:
def get_L_k(C_k:item_sets,data):
    for item in C_k:
        for item1 in data:
            if item.item_set.issubset(item1):
                C_k[item.item_set].num+=1
    C_k.filter(min_sup)
    return C_k


In [115]:
result = []
result.append(L_k)
print(L_k)
#L_k为空时停止
while len(L_k)>1:
    C_k = get_C_k(L_k)
    C_k = cut(C_k,L_k)
    L_k = get_L_k(C_k,data)
    print(L_k)
    result.append(L_k)

    

{1}:2
{3}:3
{2}:3
{5}:3

{1, 3}:2
{2, 3}:2
{3, 5}:2
{2, 5}:3

{2, 3, 5}:2

